In [1]:
import json
import logging
from enum import Enum
from typing import List, Dict, Union

from pydantic import BaseModel, field_validator, model_validator

from src.game_logic.board import Coord
from src.game_logic.game import Game
from src.game_logic.units import Unit, UnitType

logger = logging.getLogger(__name__)




In [8]:
Unit(
    location=Coord(row=0, col=0), owner_id=1, id=0, type=UnitType.warrior, actions=0)

Unit(location=Coord(row=0, col=0), owner_id=1, id=0, type=<UnitType.warrior: 'warrior'>, actions=0)

In [12]:
class ActionType(Enum):
    move_unit = "move_unit"
    attack = "attack"
    build_unit = "build_unit"
    end_turn = "end_turn"


class ActionParam(BaseModel):
    game_id: int
    player_id: int
    class Config:
        extra = "forbid"


class ActionParamMoveUnit(ActionParam):
    unit_id: int
    move_to: Coord


class ActionParamAttack(ActionParam):
    attacking_unit_ids: List[int]
    attacked_unit_id: int


class ActionParamBuildUnit(ActionParam):
    location: Coord
    type: UnitType


class ActionParamEndTurn(ActionParam):
    pass


class Action(BaseModel):
    action_type: ActionType
    params: Union[ActionParamMoveUnit, ActionParamAttack, ActionParamBuildUnit, ActionParamEndTurn]
    
    @field_validator("params", mode="after")
    def valid_params(cls, v,  values):
        if values.data["action_type"] == ActionType.move_unit:
            assert isinstance(v, ActionParamMoveUnit)
        elif values.data["action_type"] == ActionType.attack:
            assert isinstance(v, ActionParamAttack)
        elif values.data["action_type"] == ActionType.build_unit:
            assert isinstance(v, ActionParamBuildUnit)
        elif values.data["action_type"] == ActionType.end_turn:
            assert isinstance(v, ActionParamEndTurn)

In [2]:
a = "hello"
a[:100]

'hello'

In [13]:
data = {
    "action_type" : "move_unit",
    "params": {
        "game_id" : 0,
        'player_id': 1,
        "unit_id":0,
        "move_to":{
            "row":0,
            "col":0,
        }
    }
}
data2 = {
    "action_type" : "attack",
    "params": {
        "game_id" : 0,
        'player_id': 1,
        "attacking_unit_ids":[0,1,2],
        "attacked_unit_id":3
    }
}
data3 = {
    "action_type" : "build_unit",
    "params": {
        "game_id" : 0,
        'player_id': 1,
        "location":{
            "row":0,
            "col":0,
        },
        "type":"warrior"
    }
}
data4 = {
    "action_type" : "end_turn",
    "params": {
        "game_id" : 0,
        'player_id': 1
    }
}
print(data3)
a = [Action.model_validate(d) for d in [data, data2, data3, data4]]
print(a)

{'action_type': 'build_unit', 'params': {'game_id': 0, 'player_id': 1, 'location': {'row': 0, 'col': 0}, 'type': 'warrior'}}
[Action(action_type=<ActionType.move_unit: 'move_unit'>, params=None), Action(action_type=<ActionType.attack: 'attack'>, params=None), Action(action_type=<ActionType.build_unit: 'build_unit'>, params=None), Action(action_type=<ActionType.end_turn: 'end_turn'>, params=None)]


In [10]:
ActionParam(**data3["params"])

ValidationError: 2 validation errors for ActionParam
location
  Extra inputs are not permitted [type=extra_forbidden, input_value={'row': 0, 'col': 0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.0.1/v/extra_forbidden
type
  Extra inputs are not permitted [type=extra_forbidden, input_value='warrior', input_type=str]
    For further information visit https://errors.pydantic.dev/2.0.1/v/extra_forbidden

In [7]:
data3["params"]

{'game_id': 0,
 'player_id': 1,
 'location': {'row': 0, 'col': 0},
 'type': 'warrior'}